In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
from tracr.rasp import rasp
from tracr.compiler import compiling, rasp_to_graph
import numpy as np
from tracr.compiler.validating import validate
from typing import Union, TypeVar
from lauro_inverse_tracr import map_primitives, sampling, utils
import networkx as nx

from tracr.rasp.rasp import Map, SequenceMap, LinearSequenceMap, Select, Aggregate, Comparison, SelectorWidth, indices, tokens 


def compile_rasp_to_model(sop: rasp.SOp, vocab={1,2,3,4}, max_seq_len=5, compiler_bos="BOS"):
    return compiling.compile_rasp_to_model(
        sop,
        vocab=vocab,
        max_seq_len=max_seq_len,
        compiler_bos=compiler_bos
    )


TEST_INPUT = [1,2,3,4]

# Proposal for program generation

In [2]:
sampler = sampling.ProgramSampler(validate_compilation=True)
sampler.sample()
print(sampler.output([1,2,3,4]))

ValueError: Failed to find a node with label tokens. This is probably because your RASP program does not include rasp.tokens. A program must include rasp.tokens to be compiled.

In [5]:
for sop in sampler.sops[2:]:
    compile_rasp_to_model(sop)

ValueError: Failed to find a node with label tokens. This is probably because your RASP program does not include rasp.tokens. A program must include rasp.tokens to be compiled.

In [8]:
utils.print_program(sop)

select_2 = Select(indices, indices, predicate=Comparison.FALSE)
aggregate_1 = Aggregate(select_2, indices)    # type: categorical


In [3]:
# print program with and without test input
utils.print_program(sampler.sops[-1])
print()
print()
utils.print_program(sampler.sops[-1], TEST_INPUT)

sequence_map_7 = SequenceMap(lambda x, y: x*y, tokens, indices)    # type: categorical
select_9 = Select(sequence_map_7, tokens, predicate=Comparison.LEQ)
selector_width_6 = SelectorWidth(select_9)    # type: categorical
select_4 = Select(selector_width_6, sequence_map_7, predicate=Comparison.NEQ)
select_8 = Select(indices, selector_width_6, predicate=Comparison.EQ)
aggregate_5 = Aggregate(select_8, selector_width_6)    # type: categorical
aggregate_3 = Aggregate(select_4, aggregate_5)    # type: categorical


sequence_map_7 = SequenceMap(lambda x, y: x*y, tokens, indices)    # output: [0, 2, 6, 12]
select_9 = Select(sequence_map_7, tokens, predicate=Comparison.LEQ)
selector_width_6 = SelectorWidth(select_9)    # output: [1, 2, 2, 2]
select_4 = Select(selector_width_6, sequence_map_7, predicate=Comparison.NEQ)
select_8 = Select(indices, selector_width_6, predicate=Comparison.EQ)
aggregate_5 = Aggregate(select_8, selector_width_6)    # output: [2, 2, 2, 2]
aggregate_3 = Aggregate(select